# Max Verstappen Probability to get to current position(after Qatar GP) using Monte Carlo Method

## Lets calculate the probability that Max Verstappen still had to close the gap by atleast 92 points

In [1]:
import pandas as pd
import numpy as np
import fastf1
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
schedule_2025 = fastf1.get_event_schedule(2025, include_testing=0)
schedule_2025

req         WARNING 	DEFAULT CACHE ENABLED! (62.12 MB) C:\Users\farha\AppData\Local\Temp\fastf1


,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
1,1,Australia,Melbourne,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,2025-03-16,Australian Grand Prix,conventional,Practice 1,2025-03-14 12:30:00+11:00,2025-03-14 01:30:00,...,Practice 3,2025-03-15 12:30:00+11:00,2025-03-15 01:30:00,Qualifying,2025-03-15 16:00:00+11:00,2025-03-15 05:00:00,Race,2025-03-16 15:00:00+11:00,2025-03-16 04:00:00,True
2,2,China,Shanghai,FORMULA 1 HEINEKEN CHINESE GRAND PRIX 2025,2025-03-23,Chinese Grand Prix,sprint_qualifying,Practice 1,2025-03-21 11:30:00+08:00,2025-03-21 03:30:00,...,Sprint,2025-03-22 11:00:00+08:00,2025-03-22 03:00:00,Qualifying,2025-03-22 15:00:00+08:00,2025-03-22 07:00:00,Race,2025-03-23 15:00:00+08:00,2025-03-23 07:00:00,True
3,3,Japan,Suzuka,FORMULA 1 LENOVO JAPANESE GRAND PRIX 2025,2025-04-06,Japanese Grand Prix,conventional,Practice 1,2025-04-04 11:30:00+09:00,2025-04-04 02:30:00,...,Practice 3,2025-04-05 11:30:00+09:00,2025-04-05 02:30:00,Qualifying,2025-04-05 15:00:00+09:00,2025-04-05 06:00:00,Race,2025-04-06 14:00:00+09:00,2025-04-06 05:00:00,True
4,4,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2025,2025-04-13,Bahrain Grand Prix,conventional,Practice 1,2025-04-11 14:30:00+03:00,2025-04-11 11:30:00,...,Practice 3,2025-04-12 15:30:00+03:00,2025-04-12 12:30:00,Qualifying,2025-04-12 19:00:00+03:00,2025-04-12 16:00:00,Race,2025-04-13 18:00:00+03:00,2025-04-13 15:00:00,True
5,5,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2025,2025-04-20,Saudi Arabian Grand Prix,conventional,Practice 1,2025-04-18 16:30:00+03:00,2025-04-18 13:30:00,...,Practice 3,2025-04-19 16:30:00+03:00,2025-04-19 13:30:00,Qualifying,2025-04-19 20:00:00+03:00,2025-04-19 17:00:00,Race,2025-04-20 20:00:00+03:00,2025-04-20 17:00:00,True
6,6,United States,Miami Gardens,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2025,2025-05-04,Miami Grand Prix,sprint_qualifying,Practice 1,2025-05-02 12:30:00-04:00,2025-05-02 16:30:00,...,Sprint,2025-05-03 12:00:00-04:00,2025-05-03 16:00:00,Qualifying,2025-05-03 16:00:00-04:00,2025-05-03 20:00:00,Race,2025-05-04 16:00:00-04:00,2025-05-04 20:00:00,True
7,7,Italy,Imola,FORMULA 1 AWS GRAN PREMIO DEL MADE IN ITALY E ...,2025-05-18,Emilia Romagna Grand Prix,conventional,Practice 1,2025-05-16 13:30:00+02:00,2025-05-16 11:30:00,...,Practice 3,2025-05-17 12:30:00+02:00,2025-05-17 10:30:00,Qualifying,2025-05-17 16:00:00+02:00,2025-05-17 14:00:00,Race,2025-05-18 15:00:00+02:00,2025-05-18 13:00:00,True
8,8,Monaco,Monaco,FORMULA 1 TAG HEUER GRAND PRIX DE MONACO 2025,2025-05-25,Monaco Grand Prix,conventional,Practice 1,2025-05-23 13:30:00+02:00,2025-05-23 11:30:00,...,Practice 3,2025-05-24 12:30:00+02:00,2025-05-24 10:30:00,Qualifying,2025-05-24 16:00:00+02:00,2025-05-24 14:00:00,Race,2025-05-25 15:00:00+02:00,2025-05-25 13:00:00,True
9,9,Spain,Barcelona,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2025,2025-06-01,Spanish Grand Prix,conventional,Practice 1,2025-05-30 13:30:00+02:00,2025-05-30 11:30:00,...,Practice 3,2025-05-31 12:30:00+02:00,2025-05-31 10:30:00,Qualifying,2025-05-31 16:00:00+02:00,2025-05-31 14:00:00,Race,2025-06-01 15:00:00+02:00,2025-06-01 13:00:00,True
10,10,Canada,Montréal,FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2025,2025-06-15,Canadian Grand Prix,conventional,Practice 1,2025-06-13 13:30:00-04:00,2025-06-13 17:30:00,...,Practice 3,2025-06-14 12:30:00-04:00,2025-06-14 16:30:00,Qualifying,2025-06-14 16:00:00-04:00,2025-06-14 20:00:00,Race,2025-06-15 14:00:00-04:00,2025-06-15 18:00:00,True


In [3]:
race_data = []
# Races between Zandvoort (R15) and Qatar (R23) are Rounds 16 through 23
rounds_to_analyze = list(range(16, 24)) 

for r in rounds_to_analyze:
    try:
        # Load the race session
        session = fastf1.get_session(2025, r, 'R')
        session.load(telemetry=False, weather=False, messages=False)
        
        results = session.results
        
        # Get points for the drivers
        ver_points = results[results['Abbreviation'] == 'VER']['Points'].iloc[0]
        nor_points = results[results['Abbreviation'] == 'NOR']['Points'].iloc[0]
        pia_points = results[results['Abbreviation'] == 'PIA']['Points'].iloc[0]

        
        race_data.append({
            'Race': session.event['EventName'],
            'VER_Race_Points': ver_points,
            'NOR_Race_Points': nor_points,
            'PIA_Race_Points': pia_points 
        })
        
    except Exception as e:
        print(f"Skipping Round {r} due to error: {e}")

# Convert the list of dictionaries to a DataFrame
df_race = pd.DataFrame(race_data)

core           INFO 	Loading data for Italian Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '16', '63', '44', '23', '5', '12', '6', '55', '87', '22', '30', '31', '10', '43', '18', '14', '27']
core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for

In [4]:
df_race

,Race,VER_Race_Points,NOR_Race_Points,PIA_Race_Points
0,Italian Grand Prix,25.0,18.0,15.0
1,Azerbaijan Grand Prix,25.0,6.0,0.0
2,Singapore Grand Prix,18.0,15.0,12.0
3,United States Grand Prix,25.0,18.0,10.0
4,Mexico City Grand Prix,15.0,25.0,10.0
5,São Paulo Grand Prix,15.0,25.0,10.0
6,Las Vegas Grand Prix,25.0,0.0,0.0
7,Qatar Grand Prix,25.0,12.0,18.0


In [5]:
sprint_data = []
# Races between Zandvoort (R15) and Qatar (R23) are Rounds 16 through 23
rounds_to_analyze = list(range(16, 24)) 

SPRINT_ROUNDS = {19, 21, 23}

for r in rounds_to_analyze:
    if r in SPRINT_ROUNDS:
        try:
            # Load the race session
            session = fastf1.get_session(2025, r, 'S')
            session.load(telemetry=False, weather=False, messages=False)
            
            results = session.results
            
            # Get points for the drivers
            ver_points = results[results['Abbreviation'] == 'VER']['Points'].iloc[0]
            nor_points = results[results['Abbreviation'] == 'NOR']['Points'].iloc[0]
            pia_points = results[results['Abbreviation'] == 'PIA']['Points'].iloc[0]

            
            sprint_data.append({
                'Race': session.event['EventName'],
                'VER_Points_S': ver_points,
                'NOR_Points_S': nor_points,
                'PIA_Points_S': pia_points 
            })
        
        except Exception as e:
            print(f"Skipping Round {r} due to error: {e}")
    else:
        ver_points, nor_points, pia_points = 0.0, 0.0, 0.0
        session = fastf1.get_session(2025, r, 'R')
        session.load(telemetry=False, weather=False, messages=False)
        sprint_data.append({
                'Race': session.event['EventName'],
                'VER_Points_S': ver_points,
                'NOR_Points_S': nor_points,
                'PIA_Points_S': pia_points 
            })
    

# Convert the list of dictionaries to a DataFrame
df_sprint = pd.DataFrame(sprint_data)

core           INFO 	Loading data for Italian Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '16', '63', '44', '23', '5', '12', '6', '55', '87', '22', '30', '31', '10', '43', '18', '14', '27']
core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for

In [6]:
df_sprint

,Race,VER_Points_S,NOR_Points_S,PIA_Points_S
0,Italian Grand Prix,0.0,0.0,0.0
1,Azerbaijan Grand Prix,0.0,0.0,0.0
2,Singapore Grand Prix,0.0,0.0,0.0
3,United States Grand Prix,8.0,0.0,0.0
4,Mexico City Grand Prix,0.0,0.0,0.0
5,São Paulo Grand Prix,5.0,8.0,0.0
6,Las Vegas Grand Prix,0.0,0.0,0.0
7,Qatar Grand Prix,5.0,6.0,8.0


In [7]:
df_sprint_points_only = df_sprint[['Race', 'VER_Points_S', 'NOR_Points_S', 'PIA_Points_S']]

df_weekend = df_race.merge(
    df_sprint_points_only, 
    on='Race', 
    how='left'
)

df_weekend[['VER_Points_S', 'NOR_Points_S', 'PIA_Points_S']] = \
    df_weekend[['VER_Points_S', 'NOR_Points_S', 'PIA_Points_S']].fillna(0.0)

df_weekend['VER_Total_Points'] = df_weekend['VER_Race_Points'] + df_weekend['VER_Points_S']
df_weekend['NOR_Total_Points'] = df_weekend['NOR_Race_Points'] + df_weekend['NOR_Points_S']
df_weekend['PIA_Total_Points'] = df_weekend['PIA_Race_Points'] + df_weekend['PIA_Points_S']

In [8]:
df_weekend['Points_Difference_VER_NOR'] = df_weekend['VER_Total_Points'] - df_weekend['NOR_Total_Points']

In [9]:
df_weekend

,Race,VER_Race_Points,NOR_Race_Points,PIA_Race_Points,VER_Points_S,NOR_Points_S,PIA_Points_S,VER_Total_Points,NOR_Total_Points,PIA_Total_Points,Points_Difference_VER_NOR
0,Italian Grand Prix,25.0,18.0,15.0,0.0,0.0,0.0,25.0,18.0,15.0,7.0
1,Azerbaijan Grand Prix,25.0,6.0,0.0,0.0,0.0,0.0,25.0,6.0,0.0,19.0
2,Singapore Grand Prix,18.0,15.0,12.0,0.0,0.0,0.0,18.0,15.0,12.0,3.0
3,United States Grand Prix,25.0,18.0,10.0,8.0,0.0,0.0,33.0,18.0,10.0,15.0
4,Mexico City Grand Prix,15.0,25.0,10.0,0.0,0.0,0.0,15.0,25.0,10.0,-10.0
5,São Paulo Grand Prix,15.0,25.0,10.0,5.0,8.0,0.0,20.0,33.0,10.0,-13.0
6,Las Vegas Grand Prix,25.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,25.0
7,Qatar Grand Prix,25.0,12.0,18.0,5.0,6.0,8.0,30.0,18.0,26.0,12.0


Now extract the necessary data from the dataframe

In [ ]:
mean_point_difference = df_weekend['Points_Difference_VER_NOR'].mean()
std_point_difference = df_weekend['Points_Difference_VER_NOR'].std()
target_gap = 92 #104 - 12
no_races = 8 #Italy to Qatar
no_simulations = 100000 

In [12]:
points_diffs = np.random.normal(loc=mean_point_difference, scale=std_point_difference, size=(no_simulations, no_races))

In [ ]:
total_gains = points_diffs.sum(axis=1)
successful_simulations = (total_gains >= target_gap).sum()
probability = successful_simulations / no_simulations

In [16]:
probability

np.float64(0.18416)

In [17]:
print(f"--- Monte Carlo Simulation Results ---")
print(f"Input Distribution: Normal (mu={mean_point_difference}, sigma={std_point_difference})")
print(f"Target Gain Required: {target_gap} points over {no_races} races.")
print("-" * 40)
print(f"Total Simulations Run: {no_simulations}")
print(f"Successful Scenarios (Gain >= {target_gap} pts): {successful_simulations}")
print(f"Calculated Probability: {probability:.4f}")
print(f"The percentage chance of these events or better events occur is: {probability * 100}%")

--- Monte Carlo Simulation Results ---
Input Distribution: Normal (mu=7.25, sigma=13.424391446701996)
Target Gain Required: 92 points over 8 races.
----------------------------------------
Total Simulations Run: 100000
Successful Scenarios (Gain >= 92 pts): 18416
Calculated Probability: 0.1842
The percentage chance of these events or better events occur is: 18.416%


Therefore, the chance that Max Verstappen would have reduced the gap by 92 points was calculated to be approximately 18% To put that into perspective, the odds of this happening were approximately 1 in 5.4. This means that in nearly 82 out of every 100 instances modeled under the same performance distribution (mean = 7.25 points and standard deviation approx. = 13.42 points per race), the required gap reduction would not have been achieved. Max Verstappen has overcome massive statistical odds to have one last shot at securing his 5th consecutive WDC